# Laboratorio 3

#### Mario de León
#### Javier Valle

#### Import necesarios

In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Usando Feed Forward NN (Red simple)

In [19]:
# Cargar el dataset
data = pd.read_csv('data.csv') 


# Separando un poco el formato.
data['Year'] = data['Month'].apply(lambda x: int(x.split('-')[0]))
data['Month'] = data['Month'].apply(lambda x: int(x.split('-')[1]))

# Dividir los datos en características (X) y etiquetas (y)
X = data[['Year', 'Month']]
y = data['Sunspots']

# Conjunto de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalando los datos.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creando la estructura.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),  # 2 características de entrada (Year, Month)
    tf.keras.layers.Dense(256, activation='relu'),  
    tf.keras.layers.Dense(128, activation='relu'),  
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(32, activation='relu'),  
    tf.keras.layers.Dense(16, activation='relu'),  
    tf.keras.layers.Dense(8, activation='relu'),  
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilando el modelo.
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenando el modelo
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluando el modelo.
loss = model.evaluate(X_test_scaled, y_test)
print("Pérdida en el conjunto de prueba:", loss)

# Realiznado predicciones.
predictions = model.predict(X_test_scaled)

Epoch 1/100
71/71 [==============================] - 7s 16ms/step - loss: 4546.4067 - val_loss: 4338.5601
Epoch 2/100
71/71 [==============================] - 1s 9ms/step - loss: 3855.0554 - val_loss: 1811.6208
Epoch 3/100
71/71 [==============================] - 1s 11ms/step - loss: 1819.1913 - val_loss: 1787.5450
Epoch 4/100
71/71 [==============================] - 1s 10ms/step - loss: 1785.5952 - val_loss: 1763.0090
Epoch 5/100
71/71 [==============================] - 1s 10ms/step - loss: 1771.0082 - val_loss: 1759.8112
Epoch 6/100
71/71 [==============================] - 1s 9ms/step - loss: 1769.8899 - val_loss: 1750.1511
Epoch 7/100
71/71 [==============================] - 1s 9ms/step - loss: 1763.1667 - val_loss: 1751.3389
Epoch 8/100
71/71 [==============================] - 1s 9ms/step - loss: 1768.5315 - val_loss: 1747.0206
Epoch 9/100
71/71 [==============================] - 1s 9ms/step - loss: 1767.5894 - val_loss: 1742.8634
Epoch 10/100
71/71 [==============================]

##### Predicciones del modelo.

In [20]:
print("Predicciones: ", predictions)

Predicciones:  [[55.227173]
 [60.872795]
 [34.035294]
 [56.249866]
 [88.17943 ]
 [53.79607 ]
 [41.36508 ]
 [95.34134 ]
 [56.009155]
 [31.893251]
 [55.393543]
 [42.97194 ]
 [46.98934 ]
 [47.54872 ]
 [33.69964 ]
 [56.51853 ]
 [88.1566  ]
 [66.36552 ]
 [56.201347]
 [51.372337]
 [81.52766 ]
 [40.858154]
 [51.681473]
 [36.1304  ]
 [36.607994]
 [53.161255]
 [35.931458]
 [61.28434 ]
 [58.846924]
 [34.040432]
 [61.04664 ]
 [40.87494 ]
 [52.56299 ]
 [31.881088]
 [43.788307]
 [34.323017]
 [42.75274 ]
 [52.868546]
 [57.604073]
 [40.716267]
 [62.66231 ]
 [56.216   ]
 [78.69503 ]
 [60.855335]
 [45.55175 ]
 [57.458702]
 [51.331886]
 [29.193861]
 [30.524961]
 [60.38593 ]
 [44.451607]
 [43.455833]
 [61.30581 ]
 [59.612896]
 [84.8598  ]
 [34.39939 ]
 [32.09374 ]
 [38.761   ]
 [94.49737 ]
 [66.64712 ]
 [58.48913 ]
 [52.962143]
 [62.766556]
 [46.027298]
 [89.45785 ]
 [54.77449 ]
 [91.98978 ]
 [71.88612 ]
 [34.303387]
 [50.970036]
 [44.799126]
 [36.65245 ]
 [45.261303]
 [40.30471 ]
 [43.9731  ]
 [38.10045

## Usando RNN

In [21]:
# Cargar el conjunto de datos
data = pd.read_csv('data.csv')  

# Cambio de formato
data['Year'] = data['Month'].apply(lambda x: int(x.split('-')[0]))
data['Month'] = data['Month'].apply(lambda x: int(x.split('-')[1]))

# Preparar los datos
# Usando la secuencia como la cantidad de meses.
sequence_length = 12
input_data = []
target_data = []
for i in range(len(data) - sequence_length):
    input_sequence = data[['Year', 'Month', 'Sunspots']].iloc[i : i + sequence_length].values
    target_value = data['Sunspots'].iloc[i + sequence_length]
    input_data.append(input_sequence)
    target_data.append(target_value)

input_data = np.array(input_data)
target_data = np.array(target_data)

# Dividiendo el conjunto de datos en entrenamiento y prueba.
train_size = int(0.8 * len(input_data))
X_train, X_test = input_data[:train_size], input_data[train_size:]
y_train, y_test = target_data[:train_size], target_data[train_size:]

# Creando la estructura de la RNN
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, 3)),  # Entrada con 3 características (Year, Month, Sunspots)
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(32, activation='relu', return_sequences=True),
    tf.keras.layers.Flatten(),  # Aplananando las secuencias para la capa densa
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilando el modelo.
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenando el modelo.
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluando el modelo.
loss = model.evaluate(X_test, y_test)
print("Pérdida en el conjunto de prueba:", loss)

Epoch 1/100
71/71 [==============================] - 11s 40ms/step - loss: 18804.7031 - val_loss: 3189.3760
Epoch 2/100
71/71 [==============================] - 2s 22ms/step - loss: 959.2726 - val_loss: 1725.3877
Epoch 3/100
71/71 [==============================] - 2s 22ms/step - loss: 710.7464 - val_loss: 1170.5387
Epoch 4/100
71/71 [==============================] - 2s 22ms/step - loss: 598.7290 - val_loss: 1054.8003
Epoch 5/100
71/71 [==============================] - 2s 22ms/step - loss: 622.0212 - val_loss: 1776.8928
Epoch 6/100
71/71 [==============================] - 2s 23ms/step - loss: 562.6777 - val_loss: 1132.8196
Epoch 7/100
71/71 [==============================] - 2s 23ms/step - loss: 579.7819 - val_loss: 903.6609
Epoch 8/100
71/71 [==============================] - 2s 23ms/step - loss: 662.1356 - val_loss: 887.8067
Epoch 9/100
71/71 [==============================] - 2s 22ms/step - loss: 509.5499 - val_loss: 923.2798
Epoch 10/100
71/71 [==============================] - 2

##### Resumen del modelo

In [22]:
# Resumen del modelo para verificar la arquitectura
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 12, 64)            4352      
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 12, 64)            8256      
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 12, 32)            3104      
                                                                 
 flatten_1 (Flatten)         (None, 384)               0         
                                                                 
 dense_39 (Dense)            (None, 32)                12320     
                                                                 
 dense_40 (Dense)            (None, 1)                 33        
                                                                 
Total params: 28065 (109.63 KB)
Trainable params: 2806

## Usando LSTM

In [23]:
# Cargando el conjunto de datos
data = pd.read_csv('data.csv')

# Haciendo unos cambios al formato.
data['Year'] = data['Month'].apply(lambda x: int(x.split('-')[0]))
data['Month'] = data['Month'].apply(lambda x: int(x.split('-')[1]))

# Dividiendo los datos en características (X) y etiquetas (y)
X = data[['Year', 'Month']]
y = data['Sunspots']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalando los datos para mejorar el rendimiento de la red neuronal
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Haciendo reshape para que los datos sean adecuados para la capa LSTM (samples, timesteps, features)
X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_lstm = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# Creando la estructura de la red neuronal con LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='relu', input_shape=(1, 2), return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilando
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenando el modelo
model.fit(X_train_lstm, y_train, epochs=100, batch_size=32, validation_data=(X_test_lstm, y_test))

# Evaluando el modelo
loss = model.evaluate(X_test_lstm, y_test)
print("Pérdida en el conjunto de prueba:", loss)

Epoch 1/100
71/71 [==============================] - 19s 45ms/step - loss: 4529.8960 - val_loss: 4279.0942
Epoch 2/100
71/71 [==============================] - 1s 13ms/step - loss: 3405.8191 - val_loss: 1992.4381
Epoch 3/100
71/71 [==============================] - 1s 11ms/step - loss: 1865.4258 - val_loss: 1836.3619
Epoch 4/100
71/71 [==============================] - 1s 11ms/step - loss: 1817.5349 - val_loss: 1823.4009
Epoch 5/100
71/71 [==============================] - 1s 12ms/step - loss: 1806.1677 - val_loss: 1806.2855
Epoch 6/100
71/71 [==============================] - 1s 11ms/step - loss: 1781.7252 - val_loss: 1785.8746
Epoch 7/100
71/71 [==============================] - 1s 11ms/step - loss: 1771.1294 - val_loss: 1777.2109
Epoch 8/100
71/71 [==============================] - 1s 12ms/step - loss: 1770.3401 - val_loss: 1769.5818
Epoch 9/100
71/71 [==============================] - 1s 11ms/step - loss: 1761.2969 - val_loss: 1769.9994
Epoch 10/100
71/71 [=========================

#### Predicciones del modelo

In [26]:
# Realizando predicciones
predictions = model.predict(X_test_lstm)
print("Predicciones: ", predictions)

18/18 [==============================] - 0s 5ms/step
Predicciones:  [[47.257523]
 [59.160065]
 [43.008945]
 [47.005684]
 [86.18191 ]
 [54.394848]
 [42.00695 ]
 [94.16267 ]
 [55.20066 ]
 [43.403862]
 [46.41416 ]
 [43.48442 ]
 [46.21351 ]
 [52.174793]
 [43.546906]
 [54.71952 ]
 [86.618034]
 [63.575783]
 [55.191017]
 [45.406155]
 [79.13219 ]
 [48.266373]
 [51.946365]
 [45.173454]
 [38.150295]
 [52.643127]
 [37.937893]
 [47.46921 ]
 [47.55989 ]
 [43.23356 ]
 [47.50571 ]
 [44.71327 ]
 [52.5412  ]
 [44.170433]
 [44.903618]
 [41.281063]
 [42.507744]
 [46.202854]
 [47.318157]
 [46.34796 ]
 [47.42233 ]
 [55.84536 ]
 [76.878914]
 [59.26564 ]
 [49.051907]
 [54.831387]
 [52.50445 ]
 [36.834866]
 [43.745327]
 [47.42467 ]
 [44.47827 ]
 [48.435764]
 [46.94841 ]
 [47.615013]
 [83.65575 ]
 [40.390736]
 [43.83707 ]
 [42.989273]
 [93.25926 ]
 [64.82509 ]
 [47.319942]
 [46.37905 ]
 [60.27206 ]
 [49.65989 ]
 [86.84937 ]
 [52.545498]
 [89.5581  ]
 [67.88829 ]
 [38.965336]
 [45.471375]
 [45.779797]
 [44.0740